<a href="https://colab.research.google.com/github/Panachai-spec007/dataviz2025/blob/main/midterm2025_663020285_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ให้สืบหาฆาตรกร (10 คะแนน) และ ผู้บงการ (5 คะแนน) ในคดี ฆาตรกรรม (murder) ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City โดยเริ่มจากการตรวจสอบ ข้อมูลคดีในไฟล์ crime_scene_report.csv

In [3]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
crime_scene_report = pd.read_csv('/content/drive/MyDrive/Dataviz2025/midterm data/crime_scene_report.csv')
interview = pd.read_csv('/content/drive/MyDrive/Dataviz2025/midterm data/interview.csv')
person = pd.read_csv('/content/drive/MyDrive/Dataviz2025/midterm data/person.csv')
drivers_license = pd.read_csv('/content/drive/MyDrive/Dataviz2025/midterm data/drivers_license.csv')
get_fit_now_member = pd.read_csv('/content/drive/MyDrive/Dataviz2025/midterm data/get_fit_now_member.csv')
get_fit_now_check_in = pd.read_csv('/content/drive/MyDrive/Dataviz2025/midterm data/get_fit_now_check_in.csv')
facebook_event_checkin = pd.read_csv('/content/drive/MyDrive/Dataviz2025/midterm data/facebook_event_checkin.csv')
income = pd.read_csv('/content/drive/MyDrive/Dataviz2025/midterm data/income.csv')

In [13]:
crime_scene_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1228 non-null   int64 
 1   type         1228 non-null   object
 2   description  1228 non-null   object
 3   city         1228 non-null   object
dtypes: int64(1), object(3)
memory usage: 38.5+ KB


In [16]:
crime_report = crime_scene_report[
    (crime_scene_report['date'] == 20180115) &
    (crime_scene_report['city'] == 'Pandas City') &
    (crime_scene_report['type'] == 'murder')
]
crime_report

,date,type,description,city
1227,20180115,murder,Security footage shows that there were 2 witne...,Pandas City


หา id พยายานคนที่ 1
ได้ id = 14887

In [33]:
northwestern_dr_residents = person[person['address_street_name'] == 'Northwestern Dr']
northwestern_dr_residents_sorted = northwestern_dr_residents.sort_values(by='address_number', ascending=True)
last_house = northwestern_dr_residents_sorted.iloc[-1]

print(last_house)

id                               14887
name                    Morty Schapiro
license_id                      118009
address_number                    4919
address_street_name    Northwestern Dr
ssn                          111564949
Name: 499, dtype: object


In [35]:
print(last_house['id'])

14887


หา id พยายานคนที่ 2
ได้ id = 16371

In [28]:
witness_id = person[
    person['name'].str.contains('Annabel')&
    (person['address_street_name'] == 'Franklin Ave')
]

print(witness_id)

        id            name  license_id  address_number address_street_name  \
665  16371  Annabel Miller      490173             103        Franklin Ave   

           ssn  
665  318771143  


คำให้การของพยาน

In [46]:
witness_statements = interview[
    interview['person_id'].isin([14887, 16371])
]
witness_statements


,person_id,transcript
4988,14887,I heard a gunshot and then saw a man run out. ...
4989,16371,"I saw the murder happen, and I recognized the ..."


หาตัวฆาตกรตามคำให้การ

In [44]:
# ค้นหาสมาชิก Get Fit Now ที่มี membership_id ขึ้นต้นด้วย '48Z' ตามคำให้การของพยาน
suspect_member = get_fit_now_member[
    get_fit_now_member['id'].str.startswith('48Z')
]

# เชื่อมข้อมูลกับ get_fit_now_check_in เพื่อยืนยันการเช็คอินในวันเกิดเหตุ
suspect_check_in = get_fit_now_check_in[
    (get_fit_now_check_in['check_in_date'] == 20180115) &
    (get_fit_now_check_in['membership_id'].str.startswith('48Z'))
]

# เชื่อมข้อมูลกับ person และ drivers_license เพื่อหาชื่อและป้ายทะเบียน
suspect_person_info = pd.merge(suspect_member, person, left_on='person_id', right_on='id', how='inner')
suspect_full_info = pd.merge(suspect_person_info, drivers_license, left_on='license_id', right_on='id', how='inner')

# กรองด้วยเบาะแสป้ายทะเบียน 'H42W'
killer_info = suspect_full_info[
    suspect_full_info['plate_number'].str.contains('H42W')
]
killer_info


,id_x,person_id,name_x,membership_start_date,membership_status,id_y,name_y,license_id,address_number,address_street_name,ssn,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
1,48Z55,67318,Jeremy Bowers,20160101,gold,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


In [68]:
print(killer_info['name_x'])

1    Jeremy Bowers
Name: name_x, dtype: object


คำให้การของฆาตกร

In [54]:
killer_statements = interview[
    interview['person_id'].isin([67318])
]
killer_statements

,person_id,transcript
4990,67318,I was hired by a woman with a lot of money. I ...


หาผู้บงการจากคำให้การของฆาตกร

In [66]:
# 1. ค้นหาผู้ที่เข้าชม 'SQL Symphony Concert' 3 ครั้งในเดือนธันวาคม 2017
concert_checkins = facebook_event_checkin[
    (facebook_event_checkin['event_name'] == 'SQL Symphony Concert') &
    (facebook_event_checkin['date'] >= 20171201) &
    (facebook_event_checkin['date'] <= 20171231)
]

# นับจำนวนการเช็คอินของแต่ละคนและเลือกเฉพาะคนที่เข้า 3 ครั้ง
attendees_count = concert_checkins['person_id'].value_counts()
potential_masterminds_id = attendees_count[attendees_count == 3].index

# เชื่อมข้อมูล person_id กับ license_id
suspect_person_info = person[person['id'].isin(potential_masterminds_id)]
suspect_license_info = pd.merge(suspect_person_info, drivers_license, left_on='license_id', right_on='id')

# กรองตามคุณสมบัติที่เหลือ: เพศหญิง, ผมสีแดง, สูง 65-67 นิ้ว, ขับ Tesla Model S
mastermind = suspect_license_info[
    (suspect_license_info['gender'] == 'female') &
    (suspect_license_info['hair_color'] == 'red') &
    (suspect_license_info['height'] >= 65) &
    (suspect_license_info['height'] <= 67) &
    (suspect_license_info['car_make'] == 'Tesla') &
    (suspect_license_info['car_model'] == 'Model S')
]

mastermind


,id_x,name,license_id,address_number,address_street_name,ssn,id_y,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,red,female,500123,Tesla,Model S


สรุปชื่อฆาตกรและผู้บงการ

In [73]:
print('ชื่อฆาตกร')
print(killer_info['name_x'])
print('ชื่อผู้บงการ')
print(mastermind['name'])

ชื่อฆาตกร
1    Jeremy Bowers
Name: name_x, dtype: object
ชื่อผู้บงการ
0    Miranda Priestly
Name: name, dtype: object
